You should have Voxelizer.py in your working folder, and inside your data 'processed' folder from numan you should have the 'drift_corrected_af_to1000' folder with drift corrected truncated dataset from numan notebook #02

You have to run (and save) this multiple times for multiple ROIs!

In [ ]:
import numan as nu
import vodex as vx
import Voxelizer as voxz
import pandas as pd
import numpy as np
import os

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
from tqdm.notebook import tqdm, trange

import models, analysis

Define folders here!!

In [ ]:
# raw dataset folder (even a list of multiple fish could be ok, but you loose their identity)
base_folder = "/Users/mirckuz/Desktop/"
folders = [base_folder + "20230601_Hz09_casper_h2bcamp7f_7dpf_60Z_1hzvol_2P_1v2v3v4v5_processed"]

# .tif mask of your brain
mask_file = "/Users/mirckuz/Desktop/Voxelizer/mask.tif"
# truncated annotation (numerosity, shape, spread) compatible with drift corrected data in 'drift_corrected_af_to1000' from numan notebook #02
annotation4 = "/Users/mirckuz/Desktop/Voxelizer/stimuli_truncated_timelines.csv" 

# analyzed data saving - remeber to change name accordingly!!
save_folder = '/Users/mirckuz/Desktop/'
file_name = 'fishx_dayx_roix.csv'

Define parameters and ROI

In [ ]:
superVoxel_size = [3,3,3] # order: z,y,x
roi_id = 1 #select mask integer

frames_per_volume = 60
starting_slice = 0
n_vol_batch = 8
analyze_SV_or_Imaris = 0 #0 = SV, 1 = Imaris Spots

Run super voxel analysis analysis - it takes some time depending on recording and ROI size!

In [ ]:
annotation_reg = pd.read_csv(annotation4)

for count,f in enumerate(folders):
    print('Processing: '+f)
    os.chdir(f+'/processed/')
    #os.chdir(f)
    data_dir = 'drift_corrected_af_to1000'
    print('Loading experiment...')
    experiment = vx.Experiment.from_dir(data_dir,frames_per_volume,starting_slice,verbose=False)
    #experiment.add_annotations_from_df(annotation_df1, cycles=True)
    #experiment.add_annotations_from_df(annotation_df2, cycles=True)
    #experiment.add_annotations_from_df(annotation_df3, cycles=True)
    experiment.add_annotations_from_volume_annotation_df(annotation_reg)
    experiment.labels_df

    if analyze_SV_or_Imaris == 1:
        my_spots = nu.Spots.from_json("spots/signals/spots_SvB_max.json")
        dff = my_spots.signals.as_dff(15)
        signals = dff.traces.T
    elif analyze_SV_or_Imaris == 0:
        print('Voxelizing...')
        voxelizer = voxz.Voxelizer(mask_file, superVoxel_size, roi_id)
        table = voxelizer.process_movie(experiment, n_vol_batch)
        print('Normalizing signal...')
        #df = voxelizer.create_signal_df(table)
        df = voxelizer.create_normalized_signal_df(table,3,[0,1,2])
        signals = np.array(df).T
    #print(signals)
    print(signals.shape)
    
    print('Preparing data table...')
    stim_volumes = experiment.choose_volumes([("number","d1"), ("number","d2"), ("number","d3"),("number","d4"), ("number","d5")], logic = "or")
    stim_signal1=signals[:,stim_volumes]
    stim_signal2=signals[:,[x + 1 for x in stim_volumes]]
    stim_signal3=signals[:,[x + 2 for x in stim_volumes]]
    stim_signal4=signals[:,[x + 3 for x in stim_volumes]]
    stim_signal = (stim_signal1+stim_signal2+stim_signal3+stim_signal4)/4
    stim_signal.shape
    print('cells X trials: ' + str(stim_signal.shape)+'\n')
    
    annotation_dict2= {f"cell_{ic}": stim_signal[ic] for ic in np.arange(len(signals))}
    annotation_dict=experiment.get_volume_annotations(stim_volumes)
    annotation_dict.update(annotation_dict2)
    annotation_df=pd.DataFrame(annotation_dict)
    #print(annotation_df.shape)
    
    Hf = np.array(annotation_df.iloc[:, 4:annotation_df.shape[1]])
    if count==0:
        final_Hf = Hf
    else:
        final_Hf = np.append(final_Hf, Hf, 1)
print('Total trials X cells: ' + str(final_Hf.shape))

Q_pd = pd.factorize(annotation_df['number'], sort=True)
labelQ = Q_pd[1]
print(labelQ)
Q = np.array(Q_pd[0])
#print(Q)
print(Q.shape)

C_pd = pd.factorize((annotation_df['shape']+ annotation_df['spread']), sort=True)
#C_pd = pd.factorize((annotation_df['shape']), sort=True)
labelC = C_pd[1]
print(labelC)
C = np.array(C_pd[0])
#print(C)
print(C.shape)
print('Done! Now go on and save overall data matrix with all SVs or continue with ANOVA analysis!')
#/Volumes/Group Share/Neuroanalysis/Numerosity/1v2v3v4v5_H2B/20230207_hb02_casper_h2bcamp7f_5dpf_2P_1v2v3v4v5_1_processed

Save final data as .csv

In [ ]:
final_labels = np.stack((C,Q),axis=1)
total_df = np.concatenate((final_labels, final_Hf),axis=1)
total_df.shape

np.savetxt(save_folder+file_name, total_df, delimiter=',')